In [1]:
# --------------------------------------
# Breast Cancer Detection Challenge
# --------------------------------------

import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# ======================
# 1️⃣ Dataset directories
# ======================
train_dir = r"C:\Users\pc\Desktop\Pinktober2025\Breast Cancer Detection\train"   
test_dir = r"C:\Users\pc\Desktop\Pinktober2025\Breast Cancer Detection\test"     

In [3]:
# ======================
# 2️⃣ Data augmentation
# ======================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # split for validation
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

Found 560 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [4]:
# ======================
# 3️⃣ Pretrained Model
# ======================
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
# ======================
# 4️⃣ Compile Model
# ======================
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
# ======================
# 5️⃣ Training
# ======================
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, checkpoint]
)

c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 798ms/step - accuracy: 0.4716 - loss: 0.7603

35/35 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.4724 - loss: 0.7603 - val_accuracy: 0.5000 - val_loss: 0.6969
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 33s 929ms/step - accuracy: 0.4602 - loss: 0.7470 - val_accuracy: 0.5000 - val_loss: 0.6907
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 723ms/step - accuracy: 0.4829 - loss: 0.7286

35/35 ━━━━━━━━━━━━━━━━━━━━ 32s 926ms/step - accuracy: 0.4828 - loss: 0.7289 - val_accuracy: 0.5786 - val_loss: 0.6901
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 32s 922ms/step - accuracy: 0.4502 - loss: 0.7467 - val_accuracy: 0.5000 - val_loss: 0.6915
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 34s 960ms/step - accuracy: 0.5499 - loss: 0.7084 - val_accuracy: 0.5214 - val_loss: 0.6911
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 32s 908ms/step - accuracy: 0.5095 - loss: 0.7227 - val_accuracy: 0.5000 - val_loss: 0.6913
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 31s 890ms/step - accuracy: 0.4436 - loss: 0.7545 - val_accuracy: 0.5000 - val_loss: 0.6913
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 842ms/step - accuracy: 0.5124 - loss: 0.7318 - val_accuracy: 0.5000 - val_loss: 0.6920


In [7]:
# ======================
# 6️⃣ Predict on Test Set
# ======================
# List of test filenames
from tensorflow.keras.preprocessing import image
filenames = sorted(os.listdir(test_dir))

# Prepare image arrays
test_images = []
for file in filenames:
    img_path = os.path.join(test_dir, file)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    test_images.append(img_array)

test_images = np.array(test_images)

# Make predictions
preds = model.predict(test_images)

# Convert predictions to labels (M/N)
pred_labels = ['M' if p > 0.5 else 'N' for p in preds]

10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step


In [8]:

# ======================
# 7️⃣ Save Results
# ======================
results = pd.DataFrame({
    'image file': filenames,     
    'label': pred_labels         
})

results.to_csv('predictions_ResNet50.csv', index=False)
print("✅ Predictions saved to predictions.csv")

✅ Predictions saved to predictions.csv


### fine tuning

In [9]:
# =========================================
# 🔧 Step 1: Unfreeze the last few layers
# =========================================

# Let's unfreeze the last 20 layers of ResNet50
for layer in base_model.layers[-20:]:
    layer.trainable = True

In [10]:
# Check how many layers are trainable now
print(f"Number of trainable layers: {np.sum([layer.trainable for layer in model.layers])}")

Number of trainable layers: 25


In [11]:
# =========================================
# 🔁 Step 2: Recompile with smaller LR
# =========================================
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # smaller LR for fine-tuning
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [12]:
# =========================================
# 🧩 Step 3: Train again with early stopping
# =========================================
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

fine_tune_early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

fine_tune_checkpoint = ModelCheckpoint(
    'fine_tuned_model.h5',
    monitor='val_accuracy',
    save_best_only=True
)

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # fewer epochs since it’s fine-tuning
    callbacks=[fine_tune_early_stop, fine_tune_checkpoint]
)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 980ms/step - accuracy: 0.5276 - loss: 0.7171

35/35 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.5276 - loss: 0.7169 - val_accuracy: 0.5000 - val_loss: 0.6947
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4172 - loss: 0.7432 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.4801 - loss: 0.7177 - val_accuracy: 0.5000 - val_loss: 0.6958
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.5196 - loss: 0.6992 - val_accuracy: 0.4857 - val_loss: 0.6915
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 874ms/step - accuracy: 0.5018 - loss: 0.7017

35/35 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.5016 - loss: 0.7019 - val_accuracy: 0.5286 - val_loss: 0.6937
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.5020 - loss: 0.7089 - val_accuracy: 0.5071 - val_loss: 0.6938
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.4963 - loss: 0.7152 - val_accuracy: 0.5071 - val_loss: 0.6937
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5297 - loss: 0.7100 - val_accuracy: 0.5143 - val_loss: 0.6938
Epoch 9/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5227 - loss: 0.6932 - val_accuracy: 0.5214 - val_loss: 0.6908
Epoch 10/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 855ms/step - accuracy: 0.4630 - loss: 0.7118

35/35 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.4641 - loss: 0.7116 - val_accuracy: 0.5357 - val_loss: 0.6934


In [13]:
from tensorflow.keras.preprocessing import image
filenames = sorted(os.listdir(test_dir))

# Prepare image arrays
test_images = []
for file in filenames:
    img_path = os.path.join(test_dir, file)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    test_images.append(img_array)

test_images = np.array(test_images)

# Make predictions
preds = model.predict(test_images)
pred_labels = ['M' if p > 0.5 else 'N' for p in preds]
results.to_csv('predictions_ResNet50_FT.csv', index=False)

10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step
